# OroraTech Challenge

#### Importing Packages

In [15]:
import netCDF4 as nc
import os

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import cv2 as cv
import geopandas as gpd
import pandas as pd
import numpy.ma as ma
from shapely.geometry import Point
from keplergl import KeplerGl

#### Fetching the Data Set

In [3]:
folder_path = 'input_data'
radiance_filename = 'VNP02MOD_NRT.A2020233.1000.001.nc'
geolocation_filename = 'VNP03MOD_NRT.A2020233.1000.001.nc'

# Creating the File Path
radiance_filepath = os.path.join(folder_path, radiance_filename)
geolocation_filepath = os.path.join(folder_path, geolocation_filename)

# Fetching the data file and storing in a variable.
radiance_netcdf = nc.Dataset(radiance_filepath)
geolocation_netcdf = nc.Dataset(geolocation_filepath)

In [4]:
# Fetching the Variables from the respective group of the nc file.
m13_fire_variable = radiance_netcdf['/observation_data/M13']
latitude_variable = geolocation_netcdf['/geolocation_data/latitude']
longitude_variable = geolocation_netcdf['/geolocation_data/longitude']

# Fetching the units and the array of values.
m13_units = m13_fire_variable.units
m13_fire_array = m13_fire_variable[:]

# Fetching the array of values.
latitude_array = latitude_variable[:]
longitude_array = longitude_variable[:]

In [12]:
# Rescaling the Fire Array to display on the screen
def rescaleFrame(frame, scale):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width, height)

    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

m13_fire_array_rescaled = rescaleFrame(m13_fire_array, 0.25)

# Applying a binary threshold value.
ret, thresh1 = cv.threshold(m13_fire_array_rescaled, 0.85,1, cv.THRESH_BINARY)

# Plotting the Threshold image on the screen.
cv.imshow('Threshold-Fire',thresh1)

# Saving the image in output directory.
frame_normed = 255 * (thresh1 - thresh1.min()) / (thresh1.max() - thresh1.min())
frame_normed = np.array(frame_normed, np.int)
output_folder = 'output'
threshold_filename = 'binarythreshold_image.png'
threshold_filepath = os.path.join(output_folder, threshold_filename)
cv.imwrite(filename=threshold_filepath, img=frame_normed)
cv.waitKey(0)

C:\TheFourthReich\anaconda3\envs\gisenv\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


-1

### Steps for moving the data into DataFrame (& GeoDataFrame)

In [13]:
# Flattening the array and converting them to a list to insert them in a dataframe.
longitude_list = list(longitude_array.flatten())
latitude_list = list(latitude_array.flatten())


In [19]:
m13_fire_list = list(ma.masked_values(m13_fire_array, np.nan).flatten())

In [24]:
fire_geolocation_df = pd.DataFrame({'Fire' : m13_fire_list, 'Longitude' : longitude_list, 'Latitude' : latitude_list})
fire_geolocation_df.head()

,Fire,Longitude,Latitude
0,NaN,-139.746368,47.533764
1,NaN,-139.724014,47.534237
2,NaN,-139.701721,47.534702
3,NaN,-139.679504,47.535152
4,NaN,-139.657318,47.535599


In [25]:
fire_geolocation_df.dropna(inplace=True)
fire_geolocation_df.head()

,Fire,Longitude,Latitude
1008,0.365053,-126.349487,47.032814
1009,0.356964,-126.334282,47.031361
1010,0.365053,-126.319077,47.029892
1011,0.381229,-126.303902,47.028431
1012,0.365053,-126.288765,47.026970


In [ ]:
fire_geolocation_df['geometry'] = [Point(x,y) for x,y in zip(fire_geolocation_df['Longitude'], fire_geolocation_df['Latitude'])]
fire_geolocation_df.head()

In [ ]:
fire_geolocation_gdf = gpd.GeoDataFrame(data=fire_geolocation_df, geometry=fire_geolocation_df['geometry'])
fire_geolocation_gdf.head()

In [ ]:
fire_geolocation_gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
fire_map = KeplerGl(height=600, width=800)
fire_map.add_data(data=fire_geolocation_gdf.copy(), name = 'area_of_interest')
fire_filename = 'fire_locations.html'
fire_filepath = os.path.join(output_folder, fire_filename)
fire_map.save_to_html(file_name=fire_filepath)